
# cross-validation(交差検証)による汎化性能の評価

In [12]:
# データのロード
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()
RANDOM_SEED=71

In [5]:
# データの分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, random_state=RANDOM_SEED)


In [6]:
# training set を用いて学習
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=RANDOM_SEED)
model.fit(X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=71, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [7]:
# test set を用いて評価
score = model.score(X_test, y_test)
print('Test set score: {}'.format(score))

Test set score: 0.868421052631579


# cross-validation
    交差検証では，データセットを kk 個に分割し，モデルの訓練と評価を kk 回行います．
    得られた kk 個の評価値の平均をとった値を最終的なモデルのスコアとして扱います．
1. データを kk 個のブロックに分ける．これを分割（fold）という．
2. 最初の分割1 を test set，残りの分割2~5 を training set とし，モデルの学習と評価を行う．
3. 分割2 を test set，残りの分割1, 3~5 を training set として，モデルの学習と評価を行う．
4. この過程を，分割3, 4, 5 を test set として繰り返す．
5. 得られた kk 個の精度の平均値をモデルの評価値とする．

In [8]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(random_state=RANDOM_SEED)

##通常の評価
##score = model.score(X_test, y_test)

# 交差検証
scores = cross_val_score(model, iris.data, iris.target)

# 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))

# スコアの平均値
import numpy as np
print('Average score: {}'.format(np.mean(scores)))

Cross-Validation scores: [0.96078431 0.92156863 0.95833333]
Average score: 0.9468954248366014


# cross-validationでパラメータを細かく設定して評価を行う
* 一般に、回帰には単純な kk 分割交差検証，クラス分類には層化 kk 分割交差検証が用いられる
* パラメータ cv に何も指定しない場合はこの選択基準で分割方法が選択される

In [9]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(random_state=RANDOM_SEED)

##通常の評価
##score = model.score(X_test, y_test)

# 交差検証
scores = cross_val_score(model, # 学習モデル
                         iris.data, iris.target, #データセットとラベル
                         cv=4, # 分割数
                         scoring="accuracy" #評価指標
                        )
# 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
# スコアの平均値
print('Average score: {}'.format(np.mean(scores)))


Cross-Validation scores: [1.         0.94871795 0.86111111 1.        ]
Average score: 0.9524572649572649


## 層化 k 分割交差検証による交差検証

In [10]:
model = LogisticRegression(random_state=RANDOM_SEED)

# 層化 k 分割交差検証(分割内でクラスの比率が同じように分割される)
from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

# 交差検証
scores = cross_val_score(model, # 学習モデル
                         iris.data, iris.target, #データセットとラベル
                         cv=stratifiedkfold, # 分割数
                         scoring="accuracy" #評価指標
                        )
# 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
# スコアの平均値
print('Average score: {}'.format(np.mean(scores)))

Cross-Validation scores: [0.96078431 0.92156863 0.95833333]
Average score: 0.9468954248366014


# 複数の評価指標で計算する場合の例
ただし、後述のcross_validateができたので、この書き方しない

In [15]:

model = LogisticRegression(random_state=RANDOM_SEED)

from sklearn.model_selection import cross_val_score

for score in ["accuracy", "f1_macro", "recall_macro"]:
    scores = cross_val_score(model,
                             iris.data, iris.target,
                             cv=3,
                             scoring=score
                            )
    print(score + " : " + str(scores.mean()))
    


accuracy : 0.9468954248366014
f1_macro : 0.9467955700798839
recall_macro : 0.9468954248366014


# VerUPに伴う精度判定関数の変更
    cross_val_score　から　cross_validateへ

In [16]:
from sklearn.model_selection import cross_validate
model = LogisticRegression(random_state=RANDOM_SEED)

# 交差検証
scoring = ["accuracy", "f1_macro", "recall_macro"]
scores = cross_validate(model, iris.data, iris.target, scoring=scoring)

print(scores)
print("accuracy : ", np.mean(scores['test_accuracy']))
print("f1_macro : ", np.mean(scores['test_f1_macro']))
print("recall_macro : ", np.mean(scores['test_recall_macro']))


{'fit_time': array([0.00135803, 0.00064993, 0.00062704]), 'score_time': array([0.00141072, 0.00111485, 0.00147986]), 'test_accuracy': array([0.96078431, 0.92156863, 0.95833333]), 'train_accuracy': array([0.94949495, 0.97979798, 0.96078431]), 'test_f1_macro': array([0.96064815, 0.92156863, 0.95816993]), 'train_f1_macro': array([0.94920342, 0.97977941, 0.96064815]), 'test_recall_macro': array([0.96078431, 0.92156863, 0.95833333]), 'train_recall_macro': array([0.94949495, 0.97979798, 0.96078431])}
accuracy :  0.9468954248366014
f1_macro :  0.9467955700798839
recall_macro :  0.9468954248366014
